In [57]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt

def blur_image(image, blur_kernel_size):
    if len(image.shape) > 2 and image.shape[2] > 1:
        # If image has more than one channel, convert it to grayscale
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    elif len(image.shape) == 2:
        # If image is already grayscale, no need for conversion
        grayscale_image = image
    else:
        raise ValueError("Unsupported image format")

    # Apply blur filter
    blurred_image = cv2.GaussianBlur(grayscale_image, (blur_kernel_size, blur_kernel_size), 0)
    
    return blurred_image


def compare_binary_images(image1, image2):
    """
    Compara dos imágenes binarias y devuelve el porcentaje de similitud.

    :param image1: Primera imagen binaria.
    :param image2: Segunda imagen binaria.
    :return: Porcentaje de similitud entre las dos imágenes.
    """
    # Asegurarse de que las imágenes tengan las mismas dimensiones
    if image1.shape != image2.shape:
        raise ValueError("Las dos imágenes deben tener las mismas dimensiones.")

    # Calcular el número de píxeles coincidentes
    matching_pixels = np.sum(image1 == image2)
    total_pixels = image1.shape[0] * image1.shape[1]

    # Calcular el porcentaje de similitud
    similarity_percentage = (matching_pixels / total_pixels) * 100

    return similarity_percentage


def filtrar_color_amarillo(imagen):
    # Convertir la imagen a formato HSV (Hue, Saturation, Value)
    hsv = cv2.cvtColor(imagen, cv2.COLOR_BGR2HSV)
    
    # Definir el rango de colores amarillos en HSV
    lower_yellow = np.array([18, 100, 100])
    upper_yellow = np.array([26, 255, 255])
    
    # Aplicar el filtro para detectar solo el color amarillo
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    
    # Crear una imagen binaria utilizando la máscara
    resultado = cv2.bitwise_and(imagen, imagen, mask=mask)
    
    return mask, resultado

def remove_small_particles(binary_image, min_area):
    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create a mask to store the filtered particles
    filtered_mask = np.zeros_like(binary_image)
    
    # Iterate through contours
    for contour in contours:
        # Calculate the area of the contour
        area = cv2.contourArea(contour)
        
        # If the area is larger than the minimum area threshold, keep the contour
        if area >= min_area:
            cv2.drawContours(filtered_mask, [contour], -1, 255, cv2.FILLED)
    
    return filtered_mask





def remove_small_particles2(binary_image, min_area):
    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create lists to store positions and areas of particles
    particle_positions = []
    particle_areas = []
    
    # Create a mask to store the filtered particles
    filtered_mask = np.zeros_like(binary_image)
    
    # Iterate through contours
    for contour in contours:
        # Calculate the area of the contour
        area = cv2.contourArea(contour)
        
        # If the area is larger than the minimum area threshold, keep the contour
        if area >= min_area:
            # Get the centroid of the contour
            M = cv2.moments(contour)
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            
            # Append position and area to the lists
            particle_positions.append([cx, cy])
            particle_areas.append(area)
            
            # Draw the contour on the filtered mask
            cv2.drawContours(filtered_mask, [contour], -1, 255, cv2.FILLED)
    
    # Convert lists to NumPy arrays
    particle_positions = np.array(particle_positions)
    particle_areas = np.array(particle_areas)
    
    return filtered_mask, particle_positions, particle_areas

def expand_binary_zone(binary_image, iterations=1):
    # Definir el kernel para la operación de dilatación
    kernel = np.ones((3, 3), np.uint8)
    
    # Aplicar la operación de dilatación
    expanded_image = cv2.dilate(binary_image, kernel, iterations=iterations)
    
    return expanded_image



def shrink_binary_zone(binary_image, iterations=1):
    # Define the kernel for the erosion operation
    kernel = np.ones((3, 3), np.uint8)
    
    # Apply the erosion operation
    shrunk_image = cv2.erode(binary_image, kernel, iterations=iterations)
    
    return shrunk_image

def filtering_1(imagen, mode=0,dif=3,bordes=1):

    mask_amarillo, resultado_amarillo = filtrar_color_amarillo(imagen)
    # Mostrar las imágenes

    # Set the minimum area threshold
    height, width = mask_amarillo.shape
    min_area_threshold = int(height*width*0.005)

    # Remove small particles from the binary image
    filtered_image,areas,positions = remove_small_particles2(mask_amarillo, min_area_threshold)

    if mode!=0:
        cv2.imshow('Original', imagen)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        cv2.imshow('Máscara Amarilla', mask_amarillo)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        cv2.imshow('Filtered Binary Image', filtered_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    filtered_image=expand_binary_zone(filtered_image,dif)

    if mode!=0:
        cv2.imshow('Filtered Binary Image', filtered_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    filtered_image=shrink_binary_zone(filtered_image,dif)

    if mode!=0:
        cv2.imshow('Filtered Binary Image', filtered_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    if (bordes==1):
        bordes = cv2.Canny(filtered_image, 100, 200)
    else:
        bordes = filtered_image
    if mode!=0:
        cv2.imshow('Bordes', bordes)
        cv2.waitKey(0)
        #cv2.waitKey(0)
        cv2.destroyAllWindows()
    return bordes , filtered_image

def centroid_and_area(binary_image):
    # Encuentra los contornos en la imagen binaria
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    centroids = []
    areas = []
    
    # Itera sobre los contornos encontrados
    for contour in contours:
        # Calcula el centroide del contorno
        M = cv2.moments(contour)
        if M['m00'] != 0:  # Asegura que no hay división por cero
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            centroids.append((cx, cy))
        
        # Calcula el área del contorno
        area = cv2.contourArea(contour)
        areas.append(area)
    
    return centroids, areas




In [58]:
import cv2
import numpy as np
import math
def add_border(image, top, bottom, left, right):
    """
    Add borders of specified size to a binary image.

    Parameters:
        image: Input binary image.
        top: Size of the top border.
        bottom: Size of the bottom border.
        left: Size of the left border.
        right: Size of the right border.

    Returns:
        Image with borders added.
    """
    # Define border size for each side
    border_size = (top, bottom, left, right)

    # Create a border filled with zeros
    bordered_image = np.zeros((image.shape[0] + top + bottom, image.shape[1] + left + right), dtype=np.uint8)

    # Copy the original image to the center of the bordered image
    bordered_image[top:top + image.shape[0], left:left + image.shape[1]] = image

    return bordered_image



def linear_transform(image, transformation_matrix, output_size):
    transformed_image = cv2.warpAffine(image, transformation_matrix, output_size, flags=cv2.INTER_NEAREST)
    return transformed_image

def ScalRot_transformation(image,scale_x,scale_y,angle_a_deg,vertical_shift,horizontal_shift):
    angle_a_rad = math.radians(angle_a_deg)

    # Calculate the transformation matrix
    transformation_matrix = np.float32([
        [scale_x * math.cos(angle_a_rad)  +horizontal_shift*scale_x * math.sin(angle_a_rad)    ,   horizontal_shift*scale_y * math.cos(angle_a_rad)   -scale_y * math.sin(angle_a_rad), 30],
        [scale_x * math.sin(angle_a_rad)  +vertical_shift*scale_x * math.cos(angle_a_rad)      ,  -vertical_shift*scale_y * math.sin(angle_a_rad)   +scale_y * math.cos(angle_a_rad), 0 ]
    ])
    # Get the shape of the input image
    height, width = image.shape[:2]
    # Calculate the size of the output image to accommodate all pixels
    corners = np.array([[0, 0], [width, 0], [0, height], [width, height]])
    transformed_corners = cv2.transform(np.array([corners]), transformation_matrix)[0]
    min_x, min_y = np.min(transformed_corners, axis=0)
    max_x, max_y = np.max(transformed_corners, axis=0)
    output_size = (int(max_x - min_x), int(max_y - min_y))
    # Update the transformation matrix to translate the image to keep it within bounds
    transformation_matrix[0, 2] -= min_x
    transformation_matrix[1, 2] -= min_y
    # Apply the affine transformation
    transformed_image = linear_transform(image, transformation_matrix, output_size)

    return transformed_image

def Calculate_pixel_position(px, py, height, width, scale_x, scale_y, angle_a_deg,vertical_shift,horizontal_shift):
    angle_a_rad = math.radians(angle_a_deg)

    # Calculate the transformation matrix
    transformation_matrix = np.float32([
        [scale_x * math.cos(angle_a_rad)  +horizontal_shift*scale_x * math.sin(angle_a_rad)    ,   horizontal_shift*scale_y * math.cos(angle_a_rad)   -scale_y * math.sin(angle_a_rad), 30],
        [scale_x * math.sin(angle_a_rad)  +vertical_shift*scale_x * math.cos(angle_a_rad)      ,  -vertical_shift*scale_y * math.sin(angle_a_rad)   +scale_y * math.cos(angle_a_rad), 0 ]
    ])
    # Get the shape of the input image
    # Calculate the size of the output image to accommodate all pixels
    corners = np.array([[0, 0], [width, 0], [0, height], [width, height]])
    transformed_corners = cv2.transform(np.array([corners]), transformation_matrix)[0]
    min_x, min_y = np.min(transformed_corners, axis=0)
    max_x, max_y = np.max(transformed_corners, axis=0)
    # Apply the affine transformation
    transformed_point = cv2.transform(np.array([[[px, py]]]), transformation_matrix)[0][0]
    # Translate the image to keep it within bounds
    transformed_point[0] -= min_x
    transformed_point[1] -= min_y
    return transformed_point

In [59]:
import random
def Obj_fun(atributes,BinImage=0,BinKernel=0,mode=0):
    if (type(BinImage)==int or type(BinKernel)==int):
        print('there are no minaes')
        return 0
    [a,px,py,Ax,Ay,my,mx]=atributes
    if mode!=0:
        cv2.imshow('ker Binary Image', BinKernel)
        cv2.imshow('img Binary Image', BinImage)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


    new_img=ScalRot_transformation(BinImage,Ax,Ay,a,my,mx)
    height, width = BinKernel.shape

    if mode!=0:
        cv2.imshow('ker Binary Image', new_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


    new_img=add_border(new_img,int(height),int(height),int(width),int(width))
    H,W=BinImage.shape
    [px,py]=Calculate_pixel_position(px,py,H,W,Ax,Ay,a,my,mx)
    px=int(px)+width
    py=int(py)+height 
    if mode!=0:
        cv2.imshow('img Binary Image', new_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    new_img = new_img[py:py+height, px:px+width]
    #print(new_img.shape)
    #print(M)
    #print(py+height,px+width)
    new_img = cv2.resize(new_img, (width, height), interpolation=cv2.INTER_NEAREST)

    if mode!=0:
        cv2.imshow('img Binary Image', new_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    result = cv2.bitwise_and(new_img, new_img, mask=BinKernel)
    if mode!=0:
        cv2.imshow('diff Binary Image', result)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    return (np.sum(result))/(20*height*width)
    
def rep_fun(atributes1,atributes2):
        return (atributes1 + atributes2)/2

def mut_fun(atributes,ImgH=0,ImgW=0,KerH=0,KerW=0,no_amp=0):
    [a,px,py,Ax,Ay,my,mx]=atributes
    if (no_amp==0):
        amp = (random.random()*0.2 + 0.9)
        Ax=amp*Ax
        amp = (random.random()*0.2 + 0.9)
        Ay=amp*Ay
    else:
        amp = (random.random()*0.1 + 0.9)
        Ax=amp*Ax
        amp = (random.random()*0.1 + 0.9)
        Ay=amp*Ay
    step=random.randint(0, int(ImgW/25))-int(ImgW/50)
    px+=step
    step=random.randint(0, int(ImgH/25))-int(ImgH/50)
    py+=step
    step=random.random()*4 - 2
    a+=step

    step=random.random()*0.1 -0.05
    mx+=step

    if abs(mx)>0.5:
        mx-=step
    step=random.random()*0.1 -0.05
    my+=step

    if abs(my)>0.5:
        my-=step

    if Ax<0.001:
        Ax=0.0015
    elif Ax>(1)*(2):
        Ax=(1)*(2)
    if Ay<0.001:
        Ay=0.0015
    elif Ay>(1)*(2):
        Ay=(1)*(2)
    if px > ImgW:
        px=ImgW
    elif px < 0:
        px=0
    if py > ImgH:
        py=ImgH
    elif py < 0:
        py=0
    if a >35:
        a=35
    elif a <-35:
        a=-35
    
    return [a,px,py,Ax,Ay,my,mx]


def MakeaRandSeed(n,ImgH=0,ImgW=0,KerH=0,KerW=0):
    seed=np.zeros((n, 7))
    for i in range(0,n):
        Ax = random.random()*(1) + (1)*0.5

        Ay = random.random()*(1) + (1)*0.5

        px=random.randint(0, ImgW)

        py=random.randint(0, ImgH)

        a=random.random()*5 -2.5

        mx=random.random()*0.5 -0.25
        my=random.random()*0.5 -0.25
        seed[i][:]=[a,px,py,Ax,Ay,my,mx]


        

    return seed
    

In [60]:
import random
class GenAlg:
    def __init__(GA, Npop, Nvar,Ofun,Rfun,Mfun):
        GA.Npop=Npop
        GA.Nvar=Nvar

        GA.population = np.zeros((Npop, Nvar))
        GA.fitness=np.zeros(Npop)


        GA.ObjFun=Ofun
        GA.RepFun=Rfun
        GA.MutpFun=Mfun

        GA.temp=1
        GA.best=np.zeros(Nvar)
        GA.bestVal=-1000000000

        GA.Pmut=1
        GA.Prep=0.1


    def FitEval(GA):
        for i in range(0,GA.Npop):
            GA.fitness[i]=GA.ObjFun(GA.population[i])
            if GA.fitness[i]>GA.bestVal:
                GA.best=np.array(GA.population[i])
                GA.bestVal=GA.fitness[i]



    def EstReprod(GA):
        GA.FitEval()
        # Apply temperature scaling to the logits
        GA.fitness = GA.fitness / GA.temp

        # Compute softmax probabilities
        exp_logits = np.exp(GA.fitness)
        GA.fitness = (exp_logits / np.sum(exp_logits))*GA.Npop

    def New_pop(GA):
        GA.EstReprod()
        old_Pop = GA.population
        i=0
        n=0
        for i in range(0,GA.Npop):
            while(GA.fitness[i]>=1):
                GA.fitness[i]=GA.fitness[i]-1
                GA.population[n]=old_Pop[i]
                n=n+1


        N = GA.Npop-n
        indices_n_mayores = np.argsort(GA.fitness)[-N:]
        for i in range(0,N):
            GA.population[n]=old_Pop[indices_n_mayores[i]]
            n=n+1

        
        old_Pop = GA.population
        for i in range(0,GA.Npop):
            mate=random.randint(0, GA.Npop)-1
            ran = random.random()
            if ran<GA.Prep:
                GA.population[i]=GA.RepFun(GA.population[i],old_Pop[mate])
            ran = random.random()
            if ran<GA.Pmut:
                GA.population[i]=GA.MutpFun(GA.population[i]) 


    def emulate_Ngen(GA,n):
        for i in range(0,n):
            #print(i/n*100,'%')
            GA.New_pop()





    def setSeed(GA,seed):
        GA.population=seed

    def put_best(GA):
        GA.population[random.randint(0, GA.Npop)-1]=GA.best
        
    def put_a_Seed(GA,seed):
        GA.population[random.randint(0, GA.Npop)-1]=seed

    def setRepFun(GA,rep):
        GA.RepFun=rep

    def setObjFun(GA,obj):
        GA.ObjFun=obj

    def setMutFun(GA,mut):
        GA.MutpFun=mut

    def setTemp(GA,t):
        GA.temp=t
    
    def setPmut(GA,t):
        GA.Pmut=t

    def setPrep(GA,t):
        GA.Prep=t
    
    def reset_bestval(GA):
        GA.bestVal=-1000000000

        






            
    

In [61]:
def test(kernName,imgName):

    #extraccion del kernel
    kernel_name = kernName
    kernel = cv2.imread(kernel_name)
    bord_kernel,filt=filtering_1(kernel, dif=2)
    full_kernel,filt=filtering_1(kernel, dif=2,bordes=0)

    #extraccion de la imagen
    img_name = imgName
    img = cv2.imread(img_name)
    img = cv2.resize(img, (500, int((500/img.shape[1])*img.shape[0])), interpolation=cv2.INTER_NEAREST)
    bord_img,filt=filtering_1(img)
    full_img,filt=filtering_1(img,bordes=0)

    #parametros para la red
    population_size=25
    kh, kw = bord_kernel.shape
    ih, iw = bord_img.shape
    seed=MakeaRandSeed(100,ImgH=ih,ImgW=iw,KerH=kh,KerW=kw)

    #suavizacion de la imagen
    bord_img=expand_binary_zone(bord_img,1)
    bord_kernel=expand_binary_zone(bord_kernel,1)
    bord_img=blur_image(bord_img,7)

    #declaracion de la red 
    population=GenAlg(population_size,6, lambda x:  Obj_fun(x,BinImage=bord_img,BinKernel=bord_kernel,mode=0),rep_fun, lambda x: mut_fun(x,ImgH=ih,ImgW=iw,KerH=kh,KerW=kw))
    population.setSeed(seed)



    T=10
    population.setObjFun(lambda x: Obj_fun(x,BinImage=bord_img,BinKernel=bord_kernel,mode=0))
    population.setPrep(0.1)
    population.reset_bestval()
    while (T>0.01):
        population.setTemp(T)
        population.emulate_Ngen(5) 
        population.put_best() 
        T*=0.8
    seed=population.best


    for i in range(0,5):
        T=0.05
        population.setObjFun(lambda x: Obj_fun(x,BinImage=full_img,BinKernel=full_kernel,mode=0))
        population.setMutFun(lambda x: mut_fun(x,ImgH=ih,ImgW=iw,KerH=kh,KerW=kw,no_amp=1))
        population.setPrep(0.8)
        population.reset_bestval()
        population.setTemp(T)
        population.emulate_Ngen(20) 
        population.put_a_Seed(seed)


        T=0.1
        population.setObjFun(lambda x: Obj_fun(x,BinImage=bord_img,BinKernel=bord_kernel,mode=0))
        population.setMutFun(lambda x: mut_fun(x,ImgH=ih,ImgW=iw,KerH=kh,KerW=kw))
        population.setPrep(0.1)
        population.reset_bestval()
        while (T>0.01):
            population.setTemp(T)
            population.emulate_Ngen(6) 
            population.put_best() 
            T*=0.8
        seed=population.best

    # Suponiendo que `img` y `kernel` son las imágenes que mencionaste


    [a,px,py,Ax,Ay,my,mx]=population.best
    print(population.best)
    [px,py]=Calculate_pixel_position(px,py,ih,iw,Ax,Ay,a,my,mx)
    px=int(px)
    py=int(py)
    img2=ScalRot_transformation(img,Ax,Ay,a,my,mx)
    new_img = img2[py:py+kh, px:px+kw]
    # Mostrar la imagen original
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title('Imagen Original')

    # Mostrar la imagen recortada
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB))
    plt.title('Imagen Recortada')

    plt.show()
    return new_img
        

In [62]:
for i in range(1, 1):
    print(i)
    image2 = "signals/jmmm.jpg"
    image1 = "Rombo.jpg"  # Suponiendo que este es el nombre de la imagen fija
    
    img=test(image1, image2)
    cv2.imwrite(f"signals/xd{i}.jpg", img)


In [63]:
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
import numpy as np


def plot_square(ang, a_x, a_y, s_x, s_y):
    """
    Plot a square given its corner coordinates (a, b, c, d)
    """



    matriz_rotacion = np.array([[np.cos(ang), -np.sin(ang)],
                                [np.sin(ang), np.cos(ang)]])
    
    # Matriz de escalado
    matriz_escala = np.array([[a_x, 0],
                               [0, a_y]])
    
    # Multiplicar las matrices de rotación y escalado
    rot = np.dot(matriz_rotacion, matriz_escala)

    sheer = np.array([[1, s_x],
                        [s_y, 1]])

    # Realizar la multiplicación de las matrices
    resultado_matriz = np.dot(sheer,rot)
    
    x = [0, 1, 0, -1, 0]
    y = [0, 1, 2, 1, 0]
    tx = [0, 0, 1, 1, 0]
    ty = [0, 1, 1, 0, 0]
    for i in range(0,len(x)):
        vector = np.array([x[i], y[i]])

        # Multiplicar el vector por la matriz resultante
        resultado_vector = np.dot(resultado_matriz, vector)
        tx[i]=resultado_vector[0]
        ty[i]=resultado_vector[1]

    plt.figure(figsize=(8, 8))
    plt.plot(x, y, 'r-', label='Cuadrado original')
    plt.plot(tx, ty, 'b-', label='Cuadrado transformado')

    # Establecer los límites de los ejes para que siempre sea un cuadrado
    plt.xlim(-6, 6)
    plt.ylim(-6, 6)

    # Etiquetas y leyenda
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Rotación con escalado')
    plt.legend()

    # Mostrar el gráfico
    plt.grid(True)
    plt.show()

def main():
    interact(plot_square,
             ang=FloatSlider(min=-np.pi, max=np.pi, step=0.01, value=0),
             a_x=FloatSlider(min=0, max=5, step=0.01, value=1),
             a_y=FloatSlider(min=0, max=5, step=0.01, value=1),
             s_x=FloatSlider(min=-1, max=1, step=0.01, value=0),
             s_y=FloatSlider(min=-1, max=1, step=0.01, value=0))

if __name__ == "__main__":
    main()


interactive(children=(FloatSlider(value=0.0, description='ang', max=3.141592653589793, min=-3.141592653589793,…

In [64]:
import os

# Define the folder path
folder_path = 'sig/'

# Initialize lists to store image names and images
image_names = []
images = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is an image (you can add more image extensions if needed)
    if filename.endswith('.png') or filename.endswith('.jpg') or filename.endswith('.jpeg'):
        # Create the full path to the image file
        img_path = os.path.join(folder_path, filename)
        
        # Read the image
        img = cv2.imread(img_path)
        
        # Ensure the image was read correctly
        if img is not None:
            # Append the image name and image to the lists
            image_names.append(filename)
            images.append(img)
        else:
            print(f"Error al leer la imagen: {filename}")

# Convert the list of images to a NumPy array for further processing (optional)
images_array = np.array(images)

# Output the image names and number of images loaded
print(f"Nombres de las imágenes: {image_names}")
print(f"Total de imágenes cargadas: {len(images)}")

Nombres de las imágenes: ['sp-06.jpg', 'sp-07.jpg', 'sp-08.jpg', 'sp-09.jpg', 'sp-12.jpg', 'sp-14.jpg', 'sp-15.jpg', 'sp-17.jpg', 'sp-19.jpg', 'sp-20.jpg', 'sp-25.jpg']
Total de imágenes cargadas: 11


In [69]:

img , _=filtrar_color_amarillo(images[0])
img2 , _=filtrar_color_amarillo(images[1])
cv2.imshow('Original', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow('Original', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(compare_binary_images(img,img2))


84.31573957016435
